<a href="https://colab.research.google.com/github/Thandeka20/High-spatial-resolution-imagery-vs-high-spectral-imagery/blob/main/Code/Sentinel_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import ee

In [43]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='thandeka-skosana')

In [44]:
#import modules
!pip install rasterio
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
import geemap
import re
import os
from IPython.display import Image, display
from google.colab import drive

**save_df_to_drive**
Function that saves a Pandas DataFrame as a csv file to a Google Drive folder specified by a google drive file path

In [45]:

def save_df_to_drive(df, file_path_in_drive):
    """
    Function that saves a Pandas DataFrame as a csv file to a Google Drive folder specified by a Google Drive file path

    parameters:
      df (pd.DataFrame): The Pandas DataFrame needing to be saved
      file_path_in_drive (str): Google Drive folder file path that will store the CSV file
    """
    try:
        # Ensure the destination directory exists
        drive.mount('/content/drive')  # Mount Google Drive to access files

        # Get the absolute path in Google Drive
        absolute_path_in_drive = os.path.join('/content/drive/My Drive', file_path_in_drive)

        # Ensure the destination directory exists in Google Drive
        os.makedirs(os.path.dirname(absolute_path_in_drive), exist_ok=True)

        # Save the DataFrame to the specified file path in Google Drive
        df.to_csv(absolute_path_in_drive, index=False)

        print(f"DataFrame saved to Google Drive at '{absolute_path_in_drive}'")

    except Exception as e:
        print("An error occurred:", str(e))

# Example usage:
# Assuming your file_path_in_drive is 'YourFolder/YourFile.csv'
# Make sure to change it according to your actual folder structure
#save_df_to_drive(your_dataframe, 'YourFolder/YourFile.csv')


**Image collection**
get_sentinel_images_ee
Function that allows the download of Sentinel-2 images from GEE

In [46]:
#import ee
#from datetime import datetime

#def get_sentinel_images_ee(latitude, longitude, start_date, end_date):
    #"""
    #Function to search and download Sentinel-2 images using Google Earth Engine.

    #Parameters:
        #latitude (float): Latitude coordinate of the desired location.
        #longitude (float): Longitude coordinate of the desired location.
        #start_date (str): Start date of the date range in the format 'YYYY-MM-DD'.
        #end_date (str): End date of the date range in the format 'YYYY-MM-DD'.

    #Returns:
        #List of image collection IDs for the retrieved Sentinel-2 images.
    #"""
    # Create a point geometry representing the location
    #point = ee.Geometry.Point(longitude, latitude)

    # Define the date range
    #date_range = ee.DateRange(datetime.strptime(start_date, '%Y-%m-%d'), datetime.strptime(end_date, '%Y-%m-%d'))

    # Filter Sentinel-2 data based on location and date range
    #collection = (ee.ImageCollection('COPERNICUS/S2')
                  #.filterBounds(point)
                  #.filterDate(date_range))

    # Get a list of image IDs in the collection
    #image_ids = collection.aggregate_array('system:id').getInfo()

    #return image_ids

# Example usage:
#latitude = -25.12  # Hazyview
#longitude = 30.88  # Hazyview
#start_date = '2023-07-27'
#end_date = '2023-07-28'

#result = get_sentinel_images_ee(latitude, longitude, start_date, end_date)
#print("Image Collection IDs:", result)

In [70]:
def get_sentinel2_bands(image_id, latitude, longitude):
    """
    Function that extracts the multispectral bands from a Sentinel-2 satellite image of a specified location.

    Parameters:
        image_id (str): Unique Sentinel-2 image ID.
        latitude (float): Latitude coordinate of the desired location.
        longitude (float): Longitude coordinate of the desired location.

    Returns:
        band_dict (dict): Dictionary of multispectral band values for a specified location with labels B1 - B12.
    """
    try:
        # Load the Sentinel-2 image by its ID
        sentinel2_image = ee.Image(image_id)

        # Rescale image bands / 10,000
        sentinel2_image = sentinel2_image.divide(10000)

        # Define a point geometry for the specified latitude and longitude
        point_geometry = ee.Geometry.Point([longitude, latitude])

        # Use the .sample() method to extract pixel values at the specified geometry
        # This will create a feature collection containing the pixel values
        pixel_values = sentinel2_image.sample(point_geometry, 10)  # 10 meters scale for Sentinel-2

        # Initialize an empty dictionary to store the band values
        band_dict = {}

        # Extract band values from the feature collection and add them to the dictionary
        for band_name in sentinel2_image.bandNames().getInfo():
            band_value = pixel_values.first().get(band_name).getInfo()
            band_dict[band_name] = band_value

        return band_dict

    except ee.EEException as e:
        return {"error": "An Earth Engine exception occurred: " + str(e)}
    except Exception as e:
        return {"error": "An unexpected error occurred: " + str(e)}

# -----------------------------------------------------------------------------------------------------------------
# Example usage:
sentinel2_image_id = 'COPERNICUS/S2/20230727T073621_20230727T075808_T36JTT'
latitude = -25.12
longitude = 30.88

band_data = get_sentinel2_bands(sentinel2_image_id, latitude, longitude)
print(band_data)


{'B1': 0.2078000009059906, 'B2': 0.18129999935626984, 'B3': 0.1673000007867813, 'B4': 0.15109999477863312, 'B5': 0.17949999868869781, 'B6': 0.27399998903274536, 'B7': 0.3231000006198883, 'B8': 0.3228999972343445, 'B8A': 0.328900009393692, 'B9': 0.18970000743865967, 'B10': 0.10100000351667404, 'B11': 0.22550000250339508, 'B12': 0.16990000009536743, 'QA10': 0, 'QA20': 0, 'QA60': 0}


In [71]:
#visualize the Sentinel-2 scene
# Define the bands to use for visualization
bands = ee.List(["B4", "B3", "B2"])

# Define visualization parameters
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
}

# Example: Display an image using the defined visualization parameters
image = ee.Image('COPERNICUS/S2/20230727T073621_20230727T075808_T36JTT')
display_image = Image(url=image.visualize(**vis_params).getThumbURL({'dimensions': '500x500'}))
display(display_image)

Extracting Sentinel-2 bands


**Calculating indices**

In [48]:
# Load the Sentinel-2 image using the provided image_id
image = ee.Image('COPERNICUS/S2/20230727T073621_20230727T075808_T36JTT')

# Adding Indices

# 1. NDVI (Normalized Difference Vegetation Index)
NDVI = image.expression('(NIR - RED) / (NIR + RED)', {
    'NIR': image.select('B8'),
    'RED': image.select('B4')
}).rename('NDVI')

# 2. Chlogreen (Chlorophyll Green Index)
Chlogreen = image.expression('(NIRnarrow) / (Green + REDedge1)', {
    'NIRnarrow': image.select('B8A'),
    'Green': image.select('B3'),
    'REDedge1': image.select('B5')
}).rename('Chlogreen')

# 3. LAnthoC (Leaf Anthocyanin Content)
LAnthoC = image.expression('(REDedge3) / (Green + REDedge1)', {
    'REDedge3': image.select('B7'),
    'Green': image.select('B3'),
    'REDedge1': image.select('B5')
}).rename('LAnthoC')

# 4. LChloC (Leaf Chlorophyll Content)
LChloC = image.expression('(REDedge3) / (REDedge1)', {
    'REDedge3': image.select('B7'),
    'REDedge1': image.select('B5')
}).rename('LChloC')

# 5. LCaroC (Leaf Carotenoid Content)
LCaroC = image.expression('(REDedge3) / (Blue - REDedge1)', {
    'REDedge3': image.select('B7'),
    'Blue': image.select('B2'),
    'REDedge1': image.select('B5')
}).rename('LCaroC')

# 6. BAI (Built-up Area Index)
BAI = image.expression('(Blue - NIR) / (Blue + NIR)', {
    'Blue': image.select('B2'),
    'NIR': image.select('B8')
}).rename('BAI')

# 7. GI (Grazing index)
GI = image.expression('(Green / Red)', {
    'Green': image.select('B3'),
    'Red': image.select('B4')
}).rename('GI')

# 8. gNDVI (Green Normalized Difference Vegetation)
gNDVI = image.expression('(NIR - Green) / (NIR + Green)', {
    'Green': image.select('B3'),
    'NIR': image.select('B8')
}).rename('gNDVI')

# 9. MSI (Multispectral Instrument)
MSI = image.expression('(SWIR1 / NIR)', {
    'SWIR1': image.select('B11'),
    'NIR': image.select('B8')
}).rename('MSI')

# 10. NDrededgeSWIR
NDrededgeSWIR = image.expression('(Rededge2 - SWIR2) / (Rededge2 + SWIR2)', {
    'Rededge2': image.select('B6'),
    'SWIR2': image.select('B12')
}).rename('NDrededgeSWIR')

# 11. NDTI (Normalized Difference Tillage Index)
NDTI = image.expression('(SWIR1 - SWIR2) / (SWIR1 + SWIR2)', {
    'SWIR1': image.select('B11'),
    'SWIR2': image.select('B12')
}).rename('NDTI')

# 12. NDVIre (Red-edge normalized difference vegetation index)
NDVIre = image.expression('(NIR - Rededge1) / (NIR + Rededge1)', {
    'NIR': image.select('B8'),
    'Rededge1': image.select('B5')
}).rename('NDVIre')

# 13. NDVI1
NDVI1 = image.expression('(NIR - SWIR1) / (NIR + SWIR1)', {
    'NIR': image.select('B8'),
    'SWIR1': image.select('B11')
}).rename('NDVI1')

# 14. NDVI2
NDVI2 = image.expression('(Green - NIR) / (Green + NIR)', {
    'NIR': image.select('B8'),
    'Green': image.select('B3')
}).rename('NDVI2')

# 15. NHI (Normalized Humidity Index)
NHI = image.expression('(SWIR1 - Green) / (SWIR1 + Green)', {
    'SWIR1': image.select('B11'),
    'Green': image.select('B3')
}).rename('NHI')

# 16. EVI (Enhanced Vegetation Index 1)
EVI = image.expression('2.5 * ((NIR - Red) / (NIR + 6 * Red - 7.5 * Blue) + 1)', {
    'NIR': image.select('B8'),
    'Red': image.select('B4'),
    'Blue': image.select('B2')
}).rename('EVI')

# 17. EVI2 (Enhanced Vegetation Index 2)
EVI2 = image.expression('2.4 * ((NIR - Red) / (NIR + Red + 1))', {
    'NIR': image.select('B8'),
    'Red': image.select('B4')
}).rename('EVI2')

# 18. EVI2_2 (2-band Enhanced Vegetation Index)
EVI2_2 = image.expression('2.5 * ((NIR - Red) / (NIR + 2.4 * Red + 1))', {
    'NIR': image.select('B8'),
    'Red': image.select('B4')
}).rename('EVI2_2')

# 19. MSAVI (Modified Soil Adjusted Vegetation Index)
MSAVI = image.expression('(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - Red))) / 2', {
    'NIR': image.select('B8'),
    'Red': image.select('B4')
}).rename('MSAVI')

# 20. Norm_G (Normalized Green)
Norm_G = image.expression('(Green) / (NIRwide + Red + Green)', {
    'NIRwide': image.select('B8'),
    'Green': image.select('B3'),
    'Red': image.select('B4')
}).rename('Norm_G')

# 21. Norm-NIR (Normalized NIR)
Norm_NIR = image.expression('(NIRwide) / (NIRwide + Red + Green)', {
    'NIRwide': image.select('B8'),
    'Green': image.select('B3'),
    'Red': image.select('B4')
}).rename('Norm_NIR')

# 22. Norm-R (Normalized Red)
Norm_Red = image.expression('(Red) / (NIRwide + Red + Green)', {
    'NIRwide': image.select('B8'),
    'Green': image.select('B3'),
    'Red': image.select('B4')
}).rename('Norm_Red')

# 23. RededgePeakArea (Red-edge peak area)
RededgePeakArea = image.expression('(Red + Rededge1 + Rededge2 + Rededge3 + NIRnarrow)', {
    'NIRnarrow': image.select('B8A'),
    'Rededge1': image.select('B5'),
    'Rededge2': image.select('B6'),
    'Rededge3': image.select('B7'),
    'Red': image.select('B4')
}).rename('RededgePeakArea')

# 24. RedSWIR1 (Bands difference)
RedSWIR1 = image.expression('(Red - SWIR)', {
    'SWIR': image.select('B11'),
    'Red': image.select('B4')
}).rename('RedSWIR1')

# 25. RTVIcore (Red-edge Triangular Vegetation Index)
RTVIcore = image.expression('(100 * (NIRnarrow - Rededge1) - 10 * (NIRnarrow - Green))', {
    'NIRnarrow': image.select('B8A'),
    'Rededge1': image.select('B5'),
    'Green': image.select('B3')
}).rename('RTVIcore')

# 26. SAVI (Soil Adjusted Vegetation Index)
SAVI = image.expression('((NIRnarrow - Red) / (NIRnarrow + Red + 0.5) * 1.5)', {
    'NIRnarrow': image.select('B8A'),
    'Red': image.select('B4')
}).rename('SAVI')

# 27. SR-BlueRededge1 (Simple Blue and Red-edge 1 Ratio)
SRBlueRededge1 = image.expression('(Blue / Rededge1)', {
    'Blue': image.select('B2'),
    'Rededge1': image.select('B5')
}).rename('SRBlueRededge1')

# 28. SR-BlueRededge2 (Simple Blue and Red-edge 2 Ratio)
SRBlueRededge2 = image.expression('(Blue / Rededge2)', {
    'Blue': image.select('B2'),
    'Rededge2': image.select('B6')
}).rename('SRBlueRededge2')

# 29. SR-BlueRededge3 (Simple Blue and Red-edge 3 Ratio)
SRBlueRededge3 = image.expression('(Blue / Rededge3)', {
    'Blue': image.select('B2'),
    'Rededge3': image.select('B7')
}).rename('SRBlueRededge3')

# 30. SR-NIRnarrowBlue (Simple ratio NIR narrow and Blue)
SRNIRnarrowBlue = image.expression('(NIRnarrow / Blue)', {
    'NIRnarrow': image.select('B8A'),
    'Blue': image.select('B2')
}).rename('SRNIRnarrowBlue')

# 31. SR-NIRnarrowGreen (Simple ratio NIR narrow and Green)
SRNIRnarrowGreen = image.expression('(NIRnarrow / Green)', {
    'NIRnarrow': image.select('B8A'),
    'Green': image.select('B3')
}).rename('SRNIRnarrowGreen')

# 32. SR-NIRnarrowRed (Simple ratio NIR narrow and Red)
SRNIRnarrowRed = image.expression('(NIRnarrow / Red)', {
    'NIRnarrow': image.select('B8A'),
    'Red': image.select('B4')
}).rename('SRNIRnarrowRed')

# 33. SR-NIRnarrowRededge1 (Simple NIR and Red-edge 1 Ratio)
SRNIRnarrowRededge1 = image.expression('(NIRnarrow / Rededge1)', {
    'NIRnarrow': image.select('B8A'),
    'Rededge1': image.select('B5')
}).rename('SRNIRnarrowRededge1')

# 34. SR-NIRnarrowRededge2 (Simple NIR and Red-edge 2 Ratio)
SRNIRnarrowRededge2 = image.expression('(NIRnarrow / Rededge2)', {
    'NIRnarrow': image.select('B8A'),
    'Rededge2': image.select('B6')
}).rename('SRNIRnarrowRededge2')

# 35. SR-NIRnarrowRededge3 (Simple NIR and Red-edge 3 Ratio)
SRNIRnarrowRededge3 = image.expression('(NIRnarrow / Rededge3)', {
    'NIRnarrow': image.select('B8A'),
    'Rededge3': image.select('B7')
}).rename('SRNIRnarrowRededge3')

# 36. STI (Soil Tillage Index)
STI = image.expression('(SWIR1 / SWIR2)', {
    'SWIR1': image.select('B11'),
    'SWIR2': image.select('B12')
}).rename('STI')

# 37. WBI (Water Body Index)
WBI = image.expression('(Blue - Red) / (Blue + Red)', {
    'Blue': image.select('B2'),
    'Red': image.select('B4')
}).rename('WBI')

# 38. NDMI (Normalized Difference Moisture Index)
NDMI = image.expression('(NIR - SWIR) / (NIR + SWIR)', {
    'NIR': image.select('B8'),
    'SWIR': image.select('B11')
}).rename('NDMI')

# 39. NDBR (Normalized Difference Burning Ratio) (also referred to as NBR)
NDBR = image.expression('(NIR - MIR) / (NIR + MIR)', {
    'NIR': image.select('B8'),
    'MIR': image.select('B12')
}).rename('NDBR')


Displaying the results

In [49]:
# Displaying the results
print(NDVI.getInfo())
print(Chlogreen.getInfo())
print(LAnthoC.getInfo())
print(LChloC.getInfo())
print(LCaroC.getInfo())
print(BAI.getInfo())
print(GI.getInfo())
print(gNDVI.getInfo())
print(MSI.getInfo())
print(NDrededgeSWIR.getInfo())
print(NDTI.getInfo())
print(NDVIre.getInfo())
print(NDVI1.getInfo())
print(NDVI2.getInfo())
print(NHI.getInfo())
print(EVI.getInfo())
print(EVI2.getInfo())
print(EVI2_2.getInfo())
print(MSAVI.getInfo())
print(Norm_G.getInfo())
print(Norm_NIR.getInfo())
print(Norm_Red.getInfo())
print(RededgePeakArea.getInfo())
print(RedSWIR1.getInfo())
print(RTVIcore.getInfo())
print(SAVI.getInfo())
print(SRBlueRededge1.getInfo())
print(SRBlueRededge2.getInfo())
print(SRBlueRededge3.getInfo())
print(SRNIRnarrowBlue.getInfo())
print(SRNIRnarrowGreen.getInfo())
print(SRNIRnarrowRed.getInfo())
print(SRNIRnarrowRededge1.getInfo())
print(SRNIRnarrowRededge2.getInfo())
print(SRNIRnarrowRededge3.getInfo())
print(STI.getInfo())
print(WBI.getInfo())
print(NDMI.getInfo())
print(NDBR.getInfo())

{'type': 'Image', 'bands': [{'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32736', 'crs_transform': [10, 0, 199980, 0, -10, 7300000]}]}
{'type': 'Image', 'bands': [{'id': 'Chlogreen', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [5490, 5490], 'crs': 'EPSG:32736', 'crs_transform': [20, 0, 199980, 0, -20, 7300000]}]}
{'type': 'Image', 'bands': [{'id': 'LAnthoC', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [5490, 5490], 'crs': 'EPSG:32736', 'crs_transform': [20, 0, 199980, 0, -20, 7300000]}]}
{'type': 'Image', 'bands': [{'id': 'LChloC', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [5490, 5490], 'crs': 'EPSG:32736', 'crs_transform': [20, 0, 199980, 0, -20, 7300000]}]}
{'type': 'Image', 'bands': [{'id': 'LCaroC', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [5490, 5490], 'crs': 'EPSG:32736', 'crs_transform': [20, 0,

Adding indicies to image

In [50]:
# Adding bands to the image
image_final = (image.addBands([
    NDVI, Chlogreen, LAnthoC, LChloC, LCaroC, BAI, GI, gNDVI, MSI, NDrededgeSWIR,
    NDTI, NDVIre, NDVI1, NDVI2, NHI, EVI, EVI2, EVI2_2, MSAVI, Norm_G, Norm_NIR,
    Norm_Red, RededgePeakArea, RedSWIR1, RTVIcore, SAVI, SRBlueRededge1, SRBlueRededge2,
    SRBlueRededge3, SRNIRnarrowBlue, SRNIRnarrowGreen, SRNIRnarrowRed, SRNIRnarrowRededge1,
    SRNIRnarrowRededge2, SRNIRnarrowRededge3, STI, WBI, NDMI, NDBR
]))

# Print the image with added bands
print(image_final.getInfo())

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [1830, 1830], 'crs': 'EPSG:32736', 'crs_transform': [60, 0, 199980, 0, -60, 7300000]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32736', 'crs_transform': [10, 0, 199980, 0, -10, 7300000]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32736', 'crs_transform': [10, 0, 199980, 0, -10, 7300000]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32736', 'crs_transform': [10, 0, 199980, 0, -10, 7300000]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [5490, 5490], 'crs': 'EPSG:32736', 'crs_transform': [20, 0, 199980, 0, -20, 7300000

Maybe turn data into a panda datasheet???

Importing training data
Upload the shapefile as a zipped folder into google colab

Loading training data from GEE

In [ ]:

# Load the training data (e.g., a FeatureCollection) from GEE
training_data = ee.FeatureCollection('projects/mapwaps-sabiecroc/assets/Training_setSc')

# Convert the training data to a GeoDataFrame
features = training_data.getInfo()['features']
training_subset = gpd.GeoDataFrame.from_features(features)

# Display the GeoDataFrame
print(training_subset.head())



In [75]:
#select bands
bands= ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8','B8A', 'B9', 'B11', 'B12','NDVI','LAnthoC', 'LChloC', 'LCaroC', 'BAI', 'GI', 'gNDVI', 'MSI', 'NDrededgeSWIR', 'NDTI',
'NDVIre', 'NDVI1', 'NDVI2', 'NHI', 'EVI', 'EVI2', 'EVI2_2','MSAVI', 'Norm_G', 'Norm_NIR', 'Norm_Red', 'RededgePeakArea', 'RedSWIR1', 'RTVIcore', 'SAVI', 'SRBlueRededge1', 'SRBlueRededge2', 'SRBlueRededge3',
'SRNIRnarrowBlue', 'SRNIRnarrowGreen', 'SRNIRnarrowRed', 'SRNIRnarrowRededge1', 'SRNIRnarrowRededge2', 'SRNIRnarrowRededge3', 'STI', 'WBI', 'NDMI', 'NDBR',];

# Sample regions from the image at training feature locations
training_subset = image_final.select(bands).sampleRegions(
    collection=training_data,
    properties=['ID', 'LULC_Class'],
    scale=10
)

Classification using random forest

In [76]:
#Classification
classifier = ee.Classifier.smileRandomForest(100).train(
    features=training_subset,
    classProperty='ID',
    inputProperties=bands
)
classified = image_final.select(bands).classify(classifier)

Defining a colour palette for the classified map

In [79]:
# Define the palette
SabieCrocPalette = [
    '351C75', 'F91DF9', '980A7D', '741b47', 'fd0618', 'E06666', 'ffcc99', 'ffffff', '999999', 'a8a800',
    '6aa84f', '14870e', 'DB992D', 'ff7f00', '000000', '0a14f9', '08f3e4'
]

# Visualization parameters
viz = {
    'min': 1,
    'max': 17,
    'palette': SabieCrocPalette
}

# Create a Map object using geemap
m = geemap.Map(center=[20, 0], zoom=3)

# Add the classified image to the map
m.addLayer(classified, viz, 'Classification')

# Display the legend
m.add_legend(title='Classification', colors=SabieCrocPalette, labels=[str(i) for i in range(1, 18)])

# Display the map
m




Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(child…